In [ ]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import sys, re, os
import numpy as np
import pandas as pd
import glob,imp
from pandas import json_normalize
import pyslha
sys.path.append('../../smodels')
from smodels.particlesLoader import getParticlesFromSLHA
pd.options.mode.chained_assignment = None #Disable copy warnings
from smodels.experiment.databaseObj import Database
from smodels.base.model import Model
from smodels.decomposition import decomposer
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.base.physicsUnits import fb, GeV
#from smodels.share.models.ppZpjj import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.smodelsLogging import setLogLevel
from smodels.base import runtime
from smodels.experiment.expSMS import ExpSMS
from smodels.experiment.defaultFinalStates import finalStates
setLogLevel("info")
#runtime.modelFile = 'smodels.share.models.ppZpjj'
!runSModelS.py -V

In [ ]:
BSMlist = getParticlesFromSLHA('TRV1bb_slha/run_10_MZp_1800_gAq_5.000000E+00.slha')  
#/home/yoxara/smodels-database/13TeV/CMS/CMS-EXO-20-008/validation/slhaFiles_Simp_Axi_new/run_05_tag_1_4th.slha
#BSMlist = getParticlesFromSLHA('TRV1bb_slha/run_95_MZp_1800_gAq_1.297300E-01.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
#model.updateParticles(inputFile=slhaFile)
print(BSMlist)

In [21]:
slhaFolder = 'TRV1bb_slha'
resultsFolder = 'TRV1bb_results'
File = 'parameters.ini'
!chmod -R +rwx {slhaFolder}

In [22]:
!runSModelS.py -p {File} -f {slhaFolder} -o {resultsFolder}

INFO in databaseObj.loadTextDatabase() in 479: Parsing text database at /home/yoxara/smodels-database/
INFO in databaseObj.createBinaryFile() in 630: /home/yoxara/smodels-database/db3.pcl created.
INFO in modelTester.loadDatabaseResults() in 497: Including non-validated results
INFO in modelTester.testPoints() in 357: Running SModelS for 1364 files with a single process. Messages will be redirected to smodels.log


In [23]:
# Convert Experimental Results list to a dictionary

data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(resultsFolder+'/*.py'):
    with open(f,'r') as ff:
        dataF = ff.read()
    dataF = dataF.replace('inf','-1')
    with open(f.replace('.py','_fix.py'),'w') as ff:
        ff.write(dataF)
    f = f.replace('.py','_fix.py')
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)

    data.append(dataDict)
    

In [24]:
print(len(data))

1364


In [25]:
#Convert data to flat DataFrame:
pd.set_option('display.float_format', '{:e}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
smodelsDF = json_normalize(data)
smodelsDF

,filename,SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_1419_MZp_6700_gAq_1.441080E-01.slha,"[{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...",7.059184e+03,0.000000e+00,8.472133e+03,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1bb_slha/run_10_MZp_1800_gAq_5.000000E+00.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1bb_slha/run_1419_MZp_6700_gAq_1.441080E-01...,3.0.0-beta,3.0.0-beta,0.000000e+00,1.412949e+03,1.014500e+00,1.070300e+00,[TRV1bb],"[(y1, 6700.0)]",CMS-EXO-20-008,None,1.300000e+01,1.380000e+02,upperLimit,1.392754e+03,1.320143e+03,"[(y1, 0.0006639051)]"
1,run_510_MZp_3300_gAq_2.500000E+00.slha,"[{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...",2.563399e+03,0.000000e+00,3.077745e+03,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1bb_slha/run_10_MZp_1800_gAq_5.000000E+00.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1bb_slha/run_510_MZp_3300_gAq_2.500000E+00....,3.0.0-beta,3.0.0-beta,0.000000e+00,5.143460e+02,1.321300e+01,1.178100e+01,[TRV1bb],"[(y1, 3300.0)]",CMS-EXO-20-008,None,1.300000e+01,1.380000e+02,upperLimit,3.892727e+01,4.365894e+01,"[(y1, 0.09821011)]"
2,run_152_MZp_6500_gAq_6.000000E+00.slha,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",1.849205e+01,0.000000e+00,2.219355e+01,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1bb_slha/run_10_MZp_1800_gAq_5.000000E+00.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1bb_slha/run_152_MZp_6500_gAq_6.000000E+00....,3.0.0-beta,3.0.0-beta,0.000000e+00,3.701505e+00,1.265000e+00,1.218800e+00,[TRV1bb],"[(y1, 6500.0)]",CMS-EXO-20-008,None,1.300000e+01,1.380000e+02,upperLimit,2.926091e+00,3.037007e+00,"[(y1, 1.116485)]"
3,run_161_MZp_2000_gAq_3.500000E+00.slha,"[{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...",1.077716e+05,0.000000e+00,1.295168e+05,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1bb_slha/run_10_MZp_1800_gAq_5.000000E+00.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1bb_slha/run_161_MZp_2000_gAq_3.500000E+00....,3.0.0-beta,3.0.0-beta,0.000000e+00,2.174518e+04,2.551300e+01,3.537500e+01,[TRV1bb],"[(y1, 2000.0)]",CMS-EXO-20-008,None,1.300000e+01,1.380000e+02,upperLimit,8.523178e+02,6.147049e+02,"[(y1, 0.1161194)]"
4,run_692_MZp_4000_gAq_2.000000E-01.slha,"[{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...",3.722035e+00,0.000000e+00,4.468088e+00,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1bb_slha/run_10_MZp_1800_gAq_5.000000E+00.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1bb_slha/run_692_MZp_4000_gAq_2.000000E-01....,3.0.0-beta,3.0.0-beta,0.000000e+00,7.460532e-01,6.654200e+00,7.906200e+00,[TRV

In [26]:
#Get SLHA data:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 5000001])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['DMINPUTS'].items()])
    #extparDict.update(dict([[str(key),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 5000001])
    BRsDict = {}
    for pdg,val in slha.decays.items():
#         if not abs(pdg) in [1000024,1000023]:
#             continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)


In [27]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [28]:
#Merge with SModelS DataFrame
dataDF = slhaDF.merge(smodelsDF,how='inner')

In [29]:
print('Final number of data points:',dataDF.shape[0])
#print(dataDF2.columns.values.tolist()) #Print all columns names

Final number of data points: 1364


In [30]:
#Save DataFrame to pickle file:
dataDF.to_csv('TRV1bb_data.csv')
print(len(dataDF))
dataDF

1364


,filename,mass.5000001,mass.5000521,width.5000001,width.5000521,extpar.1,extpar.2,extpar.3,extpar.4,xsec13TeV(fb).5000001,"BRs.y1.q,q","BRs.y1.c,c","BRs.y1.b,b","BRs.y1.t-,t+",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
0,run_1419_MZp_6700_gAq_1.441080E-01.slha,6.700000e+03,1.000000e+04,6.639051e-04,0.000000e+00,0.000000e+00,1.500000e+00,1.441080e-01,0.000000e+00,8.472133e+03,1.667766e-01,1.667766e-01,1.667761e-01,1.661176e-01,8.472133e+03,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...",7.059184e+03,0.000000e+00,8.472133e+03,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1bb_slha/run_10_MZp_1800_gAq_5.000000E+00.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1bb_slha/run_1419_MZp_6700_gAq_1.441080E-01...,3.0.0-beta,3.0.0-beta,0.000000e+00,1.412949e+03,1.014500e+00,1.070300e+00,[TRV1bb],"[(y1, 6700.0)]",CMS-EXO-20-008,None,1.300000e+01,1.380000e+02,upperLimit,1.392754e+03,1.320143e+03,"[(y1, 0.0006639051)]"
1,run_510_MZp_3300_gAq_2.500000E+00.slha,3.300000e+03,1.000000e+04,9.821011e-02,0.000000e+00,0.000000e+00,1.500000e+00,2.500000e+00,0.000000e+00,3.077745e+03,1.671198e-01,1.671196e-01,1.671178e-01,1.644032e-01,3.077745e+03,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...",2.563399e+03,0.000000e+00,3.077745e+03,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1bb_slha/run_10_MZp_1800_gAq_5.000000E+00.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1bb_slha/run_510_MZp_3300_gAq_2.500000E+00....,3.0.0-beta,3.0.0-beta,0.000000e+00,5.143460e+02,1.321300e+01,1.178100e+01,[TRV1bb],"[(y1, 3300.0)]",CMS-EXO-20-008,None,1.300000e+01,1.380000e+02,upperLimit,3.892727e+01,4.365894e+01,"[(y1, 0.09821011)]"
2,run_152_MZp_6500_gAq_6.000000E+00.slha,6.500000e+03,1.000000e+04,1.116485e+00,0.000000e+00,0.000000e+00,1.500000e+00,6.000000e+00,0.000000e+00,2.219355e+01,1.667835e-01,1.667834e-01,1.667829e-01,1.660832e-01,2.219355e+01,0.000000e+00,"[{'ID': 1, 'SMS': '(PV > y1(1)), (y1(1) > q,q)...",1.849205e+01,0.000000e+00,2.219355e+01,0.000000e+00,0.000000e+00,50,2.000000e-01,1,TRV1bb_slha/run_10_MZp_1800_gAq_5.000000E+00.slha,1.000000e+00,1.000000e-25,"eCharge,colordim",True,True,True,True,True,False,False,False,1,1,Input file ok,TRV1bb_slha/run_152_MZp_6500_gAq_6.000000E+00....,3.0.0-beta,3.0.0-beta,0.000000e+00,3.701505e+00,1.265000e+00,1.218800e+00,[TRV1bb],"[(y1, 6500.0)]",CMS-EXO-20-008,None,1.300000e+01,1.380000e+02,upperLimit,2.926091e+00,3.037007e+00,"[(y1, 1.116485)]"
3,run_161_MZp_2000_gAq_3.500000E+00.slha,2.000000e+03,1.000000e+04,1.161194e-01,0.000000e+00,0.000000e+00,1.500000e+00,3.500000e+00,0.000000e+00,1.295168e+05,1.679002e-01,1.678998e-01,1.678947e-01,1.605049e-01,1.295168e+05,0.000000e+00,"[{'ID': 1, 'SMS': '(

In [31]:
dataDF['xsec13TeV(fb).5000001'] * (dataDF['BRs.y1.b,b'])

0      1.412949e+03
1      5.143460e+02
2      3.701505e+00
3      2.174518e+04
4      7.460532e-01
           ...     
1359   3.675407e-01
1360   1.019651e+03
1361   5.809424e+02
1362   4.123791e+01
1363   1.018228e-01
Length: 1364, dtype: float64

In [32]:
dataDF['width.5000001']


0      6.639051e-04
1      9.821011e-02
2      1.116485e+00
3      1.161194e-01
4      7.625327e-04
           ...     
1359   4.150313e-01
1360   1.571866e-04
1361   1.927300e-04
1362   1.340909e-01
1363   1.313980e-04
Name: width.5000001, Length: 1364, dtype: float64

In [33]:
dataDF['ExptRes.result0.theory prediction (fb)']

0      1.412949e+03
1      5.143460e+02
2      3.701505e+00
3      2.174518e+04
4      7.460532e-01
           ...     
1359   3.675407e-01
1360   1.019651e+03
1361   5.809424e+02
1362   4.123791e+01
1363   1.018228e-01
Name: ExptRes.result0.theory prediction (fb), Length: 1364, dtype: float64

In [34]:
dataDF['ExptRes.result0.r']

0      1.392754e+03
1      3.892727e+01
2      2.926091e+00
3      8.523178e+02
4      1.121176e-01
           ...     
1359   6.245381e-01
1360   6.569493e+02
1361   1.536072e+03
1362   9.481941e+00
1363   1.861580e-02
Name: ExptRes.result0.r, Length: 1364, dtype: float64

In [35]:
dataDF['ExptRes.result0.theory prediction (fb)']/dataDF['ExptRes.result0.upper limit (fb)']

0      1.392754e+03
1      3.892727e+01
2      2.926091e+00
3      8.523176e+02
4      1.121176e-01
           ...     
1359   6.245381e-01
1360   6.569493e+02
1361   1.536072e+03
1362   9.481941e+00
1363   1.861579e-02
Length: 1364, dtype: float64

In [36]:
mask = dataDF['ExptRes.result0.r'].isna()
rows_with_nan = dataDF[mask]
rows_with_nan.to_csv('rows_with_nan_.csv', index=False)
rows_with_nan

,filename,mass.5000001,mass.5000521,width.5000001,width.5000521,extpar.1,extpar.2,extpar.3,extpar.4,xsec13TeV(fb).5000001,"BRs.y1.q,q","BRs.y1.c,c","BRs.y1.b,b","BRs.y1.t-,t+",totalxsec13TeV(fb),totalxsec8TeV(fb),SMS Decomposition,Total xsec for missing topologies (fb),Total xsec for missing topologies with displaced decays (fb),Total xsec for missing topologies with prompt decays (fb),Total xsec for topologies outside the grid (fb),OutputStatus.sigmacut,OutputStatus.minmassgap,OutputStatus.maxcond,OutputStatus.ncpus,OutputStatus.model,OutputStatus.promptwidth,OutputStatus.stablewidth,OutputStatus.eraseprompt,OutputStatus.checkinput,OutputStatus.doinvisible,OutputStatus.docompress,OutputStatus.computestatistics,OutputStatus.testcoverage,OutputStatus.combinesrs,OutputStatus.reportallsrs,OutputStatus.experimentalfeatures,OutputStatus.file status,OutputStatus.decomposition status,OutputStatus.warnings,OutputStatus.input file,OutputStatus.database version,OutputStatus.smodels version,ExptRes.result0.maxcond,ExptRes.result0.theory prediction (fb),ExptRes.result0.upper limit (fb),ExptRes.result0.expected upper limit (fb),ExptRes.result0.TxNames,ExptRes.result0.Mass (GeV),ExptRes.result0.AnalysisID,ExptRes.result0.DataSetID,ExptRes.result0.AnalysisSqrts (TeV),ExptRes.result0.lumi (fb-1),ExptRes.result0.dataType,ExptRes.result0.r,ExptRes.result0.r_expected,ExptRes.result0.Width (GeV)
